# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [51]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [52]:
city_data_df = pd.read_csv("city_data.csv")
city_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Albany,42.6001,-73.9662,296.51,89,100,1.05
1,1,Bambous Virieux,-20.3428,57.7575,293.27,62,79,7.65
2,2,Tuktoyaktuk,69.4541,-133.0374,275.09,84,5,8.49
3,3,Mataura,-46.1927,168.8643,282.86,61,20,1.23
4,4,La Rioja,42.2500,-2.5000,283.73,95,100,0.64
...,...,...,...,...,...,...,...,...
534,534,Tarakan,3.3000,117.6333,301.87,79,20,2.57
535,535,Russell,32.3502,-85.1999,296.72,58,1,2.19
536,536,Kamina,-8.7386,24.9906,288.56,62,52,2.14
537,537,Praia,14.9215,-23.5087,294.45,78,0,3.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [53]:
gmaps.configure(api_key=g_key)

In [54]:
locations = city_data_df[["Lat", "Lng"]]

humidity = city_data_df["Humidity"].astype(float)
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)
fig.add_layer(heatmap_layer)
print(fig)

Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [55]:
# Narrow down the cities to fit weather conditions
# Max Temp lower than 80 degrees but higher than 70.
# Wind speed less than 10mph
# Zero clouds
# Drop any rows that do not have all 3-conditions
# Looking for nice weather for vacation

nice_temps_df = pd.DataFrame(city_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (nice_temps_df["Max Temp"]<= 80) & (nice_temps_df["Max Temp"] > 70)
wind_speed = nice_temps_df["Wind Speed"] < 10
cloudiness = nice_temps_df["Cloudiness"]

# nice_temps_df
nice_temps_df[max_temp & wind_speed & cloudiness]
print(nice_temps_df)

                City  Max Temp  Wind Speed  Cloudiness
0             Albany    296.51        1.05         100
1    Bambous Virieux    293.27        7.65          79
2        Tuktoyaktuk    275.09        8.49           5
3            Mataura    282.86        1.23          20
4           La Rioja    283.73        0.64         100
..               ...       ...         ...         ...
534          Tarakan    301.87        2.57          20
535          Russell    296.72        2.19           1
536           Kamina    288.56        2.14          52
537            Praia    294.45        3.60           0
538      Garden City    299.81        2.55          75

[539 rows x 4 columns]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [56]:
hotel_df = pd.DataFrame(city_data_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Albany,NaN,42.6001,-73.9662,
1,Bambous Virieux,NaN,-20.3428,57.7575,
2,Tuktoyaktuk,NaN,69.4541,-133.0374,
3,Mataura,NaN,-46.1927,168.8643,
4,La Rioja,NaN,42.2500,-2.5000,
...,...,...,...,...,...
534,Tarakan,NaN,3.3000,117.6333,
535,Russell,NaN,32.3502,-85.1999,
536,Kamina,NaN,-8.7386,24.9906,
537,Praia,NaN,14.9215,-23.5087,


In [57]:
# Targeting for Hotel locations
target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type= "hotel"

# Parameters Dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearh/json"

# Use params dictionary to request a response
response = requests.get(base_url, params=params)

In [58]:
hotel_df.iloc[0][0]

'Albany'

In [59]:
# Creating a list to capture hotel names to insert into DataFrame
hotel_name =[]

# Update params dictionary at each iteration
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "Key": g_key
}

for index, row in hotel_df.iterrows():
    # Pull latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Change location with each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Request and print url for name and address of hotel
    name_address = requests.get(base_url, params=params)
    
    # Convert to json
    name_address = name_address.json()
    
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)    

In [60]:
hotel_name

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [61]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df.to_csv('hotel_export.csv')

In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [63]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))